In [1]:
%run 00_display_tables.ipynb

Successfully connected to py_brew database.
+-------------+-------------------+---------------+----------------+
| customer_id |   customer_name   | customer_city | customer_state |
+-------------+-------------------+---------------+----------------+
|     100     | Jennifer Gonzalez |   Smithland   |    Arkansas    |
+-------------+-------------------+---------------+----------------+
+---------+---------------------+------------+-------------+-------+------+
| date_id |        tmstp        |    date    | day_of_week | month | year |
+---------+---------------------+------------+-------------+-------+------+
|    1    | 2020-07-04 21:23:10 | 2020-07-04 |      4      |   7   | 2020 |
+---------+---------------------+------------+-------------+-------+------+
+------------------+------------------+---------------------+--------------+---------------+
|    product_id    | product_category | product_subcategory | product_name | product_price |
+------------------+------------------+------

#### Calculate the monthly growth rate of total sales amount for `Retail` product category over the past 12 months. Include the growth rate percentage and the corresponding month.

##### *Hint: Use window functions, date manipulations, and aggregations to calculate the monthly growth rate of sales amount. Consider partitioning the data by product category.*

- `%b` format specifier returns the abbriviated month name

In [2]:
%%sql 

SELECT
    product_category,
    DATE_FORMAT(date, '%Y-%b') AS sale_month,
    SUM(sales_amount) AS total_sales,
    LAG(SUM(sales_amount)) OVER (PARTITION BY product_category ORDER BY DATE_FORMAT(date, '%Y-%b')) AS previous_month_sales
FROM
    fact_sales
    JOIN dim_date ON fact_sales.date_id = dim_date.date_id
    JOIN dim_product ON fact_sales.product_id = dim_product.product_id
WHERE
    date >= DATE_SUB(CURDATE(), INTERVAL 12 MONTH)
    AND product_category = 'Retail'
GROUP BY product_category, DATE_FORMAT(date, '%Y-%b')
ORDER BY sale_month;

product_category,sale_month,total_sales,previous_month_sales
Retail,2022-Aug,1601.31,None
Retail,2022-Dec,2361.18,1601.31
Retail,2022-Jul,1362.36,2361.18
Retail,2022-Jun,2404.56,1362.36
Retail,2022-May,466.87,2404.56
Retail,2022-Nov,2140.65,466.87
Retail,2022-Oct,2129.45,2140.65
Retail,2022-Sep,1545.80,2129.45
Retail,2023-Apr,2364.74,1545.80
Retail,2023-Feb,1395.61,2364.74


In [18]:
%%sql 

-- using CTE incorporate the growth rate
-- move product filtering outside the CTE

WITH product_category_sales_CTE AS (
    SELECT
        product_category,
        DATE_FORMAT(date, '%Y-%b') AS sale_month,
        SUM(sales_amount) AS total_sales,
        LAG(SUM(sales_amount)) OVER (PARTITION BY product_category ORDER BY DATE_FORMAT(date, '%Y-%b')) AS previous_month_sales
    FROM
        fact_sales
        JOIN dim_date ON fact_sales.date_id = dim_date.date_id
        JOIN dim_product ON fact_sales.product_id = dim_product.product_id
    WHERE
        date >= DATE_SUB(CURDATE(), INTERVAL 12 MONTH)
    GROUP BY product_category, DATE_FORMAT(date, '%Y-%b')
    ORDER BY sale_month
)
SELECT 
    product_category,
    sale_month,
    total_sales,
    previous_month_sales,
    ROUND(((total_sales - previous_month_sales) / total_sales * 100), 2) AS growth_rate
FROM product_category_sales_CTE
WHERE product_category = 'Cold Beverages';


product_category,sale_month,total_sales,previous_month_sales,growth_rate
Cold Beverages,2022-Aug,1472.97,None,None
Cold Beverages,2022-Dec,2077.39,1472.97,29.10
Cold Beverages,2022-Jul,1412.19,2077.39,-47.10
Cold Beverages,2022-Jun,2222.83,1412.19,36.47
Cold Beverages,2022-May,555.05,2222.83,-300.47
Cold Beverages,2022-Nov,2344.26,555.05,76.32
Cold Beverages,2022-Oct,1540.79,2344.26,-52.15
Cold Beverages,2022-Sep,1443.53,1540.79,-6.74
Cold Beverages,2023-Apr,1679.94,1443.53,14.07
Cold Beverages,2023-Feb,1480.28,1679.94,-13.49
